In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

 # linear algebra
# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session=

In [ ]:
# https://www.kaggle.com/competitions/classify-fruits-fallinter-b1

In [ ]:
# IF NUMPY WARNING COMES PLEASE RUN THIS CELL A SECOND TIME

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from scipy.stats import mode 

In [ ]:
dataset = pd.read_csv("/kaggle/input/classify-fruits-fallinter-b1/fruits_train.csv")
dataset.head()

In [ ]:
testdata = pd.read_csv("/kaggle/input/classify-fruits-fallinter-b1/fruits_test.csv")
testdata.head()

In [ ]:
dataset.plot(subplots=True)

In [ ]:
# sns.displot(dataset, x="mass", hue="label")
# sns.displot(dataset, x="height", hue="label")
# sns.displot(dataset, x="width", hue="label")

In [ ]:
X = dataset.loc[:, ["mass", "width", "height"]]
Y = dataset.loc[:, "label"]
Z = testdata.loc[:, ["mass", "width", "height"]]
X.head()

In [ ]:
X.plot()

In [ ]:
def euchlidian_distance(pt1, pt2):
    dist = np.sqrt(np.sum((pt1-pt2)**2))
    return dist

In [ ]:
# to check euclidian distance function
a = np.array([82, 54, 66])
b = np.array([65, 72, 74])
print(euchlidian_distance(a, b))

In [ ]:
def KNN_scratch(X_train, Y_train, X_test, Y_test, k_val):
    y_hat = []
    
    for test_pt in X_test.to_numpy():
        distances = []
        
        for i in range(len(X_train)):
            euc_dist = euchlidian_distance(np.array(X_train.iloc[i]), test_pt)
            distances.append(euc_dist)
            
        dist_data = pd.DataFrame(data=distances, columns=["distance"], index = Y_train.index)
        
        knn_list = dist_data.sort_values(by=['distance'], axis=0)[:k_val]
        
        labels = Y_train.loc[knn_list.index]
#         print(labels)
#         voting = labels.mode()
        voting = mode(labels).mode[0]
#         print(voting)
        y_hat.append(voting)
    
    return y_hat

In [ ]:
# splitting of training data
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=0)

# considering test_size = 0 for maximum possible accracy for sample_submission data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

In [ ]:
# Z score normalization
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_sc = pd.DataFrame(scaler.fit_transform(X_train),columns = X_train.columns)
X_test_sc = pd.DataFrame(scaler.transform(X_test),columns = X_test.columns)
Z_sc = pd.DataFrame(scaler.transform(Z),columns = Z.columns)


In [ ]:
# from sklearn.preprocessing import StandardScaler
# # min max nomalisatioun
# from sklearn import preprocessing


# scaler = preprocessing.MinMaxScaler()

# X_train_sc = pd.DataFrame(scaler.fit_transform(X_train),columns = X_train.columns)
# X_test_sc = pd.DataFrame(scaler.transform(X_test),columns = X_test.columns)

# Z_sc = pd.DataFrame(scaler.transform(Z),columns = Z.columns)

In [ ]:
# from sklearn import preprocessing
# from sklearn.preprocessing import RobustScaler
# # robust nomalisatioun

# scaler = preprocessing.RobustScaler()

# X_train_sc = pd.DataFrame(scaler.fit_transform(X_train),columns = X_train.columns)
# X_test_sc = pd.DataFrame(scaler.transform(X_test),columns = X_test.columns)

# Z_sc = pd.DataFrame(scaler.transform(Z),columns = Z.columns)


In [ ]:
# to determine best value of K

accuracy_val = []

for i in range(1, 21):
    y_hat_test = KNN_scratch(X_train_sc, Y_train, X_test_sc, Y_test, i)
    accuracy_val.append(accuracy_score(Y_test, y_hat_test))
    
plt.plot(range(1, 21), accuracy_val, color='blue', marker = 'x', linestyle='dashed')
k_fval = max(accuracy_val)

for i in range(2, len(accuracy_val)):
    if accuracy_val[i] == k_fval:
        k_indx_final = i
        break;

k_indx_final = k_indx_final + 1
print(accuracy_val)
print(k_indx_final)

In [ ]:
y_hat_test = KNN_scratch(X_train_sc, Y_train, X_test_sc, Y_test, k_indx_final)
print(accuracy_score(Y_test, y_hat_test))

y_hat_test = KNN_scratch(X_train_sc, Y_train, X_test_sc, Y_test, 3)
print(accuracy_score(Y_test, y_hat_test))

y_hat_test = KNN_scratch(X_train_sc, Y_train, X_test_sc, Y_test, 5)
print(accuracy_score(Y_test, y_hat_test))

In [ ]:
y_hat_final = KNN_scratch(X_train_sc, Y_train, Z_sc, Y_test, 5)
ids = []
for i in range(1, len(y_hat_final)+1):
    ids.append(i)

print(ids)
print(y_hat_final)

In [ ]:
df = pd.DataFrame({"ID": ids,
                   "Category":y_hat_final})

In [ ]:
df.to_csv("/kaggle/working/submission_final.csv", index=False)